In [1]:
from utils import stitch_frames
import os
import random

dataset_dir = '/mnt/data1/jiali/avsbench_data/Single-source/s4_data/visual_frames'

for split in ['test', 'train', 'val']:
    base_dir = os.path.join(dataset_dir, split)
    entries = os.listdir(base_dir)
    categories = [entry for entry in entries if '.DS_Store' not in entry]
    for temp_cate in categories:
        video_list = os.listdir(os.path.join(base_dir, temp_cate))
        for temp_video in video_list:
            input_video_path = os.path.join(base_dir, temp_cate, temp_video)

            stitch_frames(base_dir, input_video_path, split, save_floder_name = '/avsbench_synthesis_visual_random4/', num_with_audio_and_mask=random.randint(1, 4))

input_image_category: mynah_bird_singing
Mixed audio saved to /mnt/data1/jiali/avsbench_synthesis_visual_random4/Single-source/s4_data/audio_wav/test/mynah_bird_singing/WNG5_IbQOLI.wav
input_image_category: mynah_bird_singing
Mixed audio saved to /mnt/data1/jiali/avsbench_synthesis_visual_random4/Single-source/s4_data/audio_wav/test/mynah_bird_singing/0tRkJ1CppTA.wav
input_image_category: mynah_bird_singing
Mixed audio saved to /mnt/data1/jiali/avsbench_synthesis_visual_random4/Single-source/s4_data/audio_wav/test/mynah_bird_singing/_8m_rszChvE.wav
input_image_category: mynah_bird_singing
Mixed audio saved to /mnt/data1/jiali/avsbench_synthesis_visual_random4/Single-source/s4_data/audio_wav/test/mynah_bird_singing/0WL1gKM13mY.wav
input_image_category: mynah_bird_singing
Mixed audio saved to /mnt/data1/jiali/avsbench_synthesis_visual_random4/Single-source/s4_data/audio_wav/test/mynah_bird_singing/PJpXG-JLEDc.wav
input_image_category: mynah_bird_singing
Mixed audio saved to /mnt/data1/ji

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data1/jiali/avsbench_data/Single-source/s4_data/visual_frames/test/train'